In [28]:
import pandas as pd
import numpy as np
import warnings
from scipy.stats import skew, kurtosis

from src.settings import ROOT_DIR

In [29]:
data: pd.DataFrame = pd.read_pickle(ROOT_DIR / 'data' / 'processed' / 'anon_imu_data_time_series_cleaned.pkl')

In [30]:
data.head()

,anon_id,date_measure,time_stamp,imu_gyroX_right,imu_gyroY_right,imu_gyroZ_right,imu_accX_right,imu_accY_right,imu_accZ_right,imu_gyroX_left,...,imu_angularX_left,imu_angularY_left,imu_angularZ_left,imu_angularX_right,imu_angularY_right,imu_angularZ_right,imu_angularX_spine,imu_angularY_spine,imu_angularZ_spine,PD
0,9,2023-9-29-11-53-21,0.0,-721.682708,15.593227,-388.127748,9.181793,2.010050,-2.132478,-786.340071,...,-0.272368,0.018669,-0.115200,-0.873084,0.076169,-0.284259,-0.598579,-0.068011,-0.241989,1
1,9,2023-9-29-11-53-21,20.0,-670.260115,106.966612,-531.556509,9.330485,2.006033,-2.221657,-836.277343,...,-0.925128,-0.000353,-0.203227,1.159180,0.022186,-0.128051,0.950772,0.073927,0.385690,1
2,9,2023-9-29-11-53-21,39.0,-1256.346678,154.354034,-583.745247,9.430883,2.003482,-2.231009,-784.334538,...,-1.895445,-0.033696,-0.329384,3.761404,-0.046249,0.084440,0.040030,-0.036652,0.075855,1
3,9,2023-9-29-11-53-21,64.0,-2645.955791,139.340391,-516.938939,9.452909,2.061261,-2.123366,-575.420332,...,-2.675410,-0.071210,-0.420512,4.947832,-0.077489,0.211777,0.153714,-0.063593,0.047170,1
4,9,2023-9-29-11-53-21,83.0,-4370.713652,81.916044,-388.833685,9.392902,2.283368,-1.919931,-228.216868,...,-2.836775,-0.099201,-0.418456,3.459541,-0.044035,0.151183,0.325708,-0.031565,0.107200,1


In [31]:
data.columns

Index(['anon_id', 'date_measure', 'time_stamp', 'imu_gyroX_right',
       'imu_gyroY_right', 'imu_gyroZ_right', 'imu_accX_right',
       'imu_accY_right', 'imu_accZ_right', 'imu_gyroX_left', 'imu_gyroY_left',
       'imu_gyroZ_left', 'imu_accX_left', 'imu_accY_left', 'imu_accZ_left',
       'imu_gyroX_spine', 'imu_gyroY_spine', 'imu_gyroZ_spine',
       'imu_accX_spine', 'imu_accY_spine', 'imu_accZ_spine',
       'imu_angleX_right', 'imu_angleY_right', 'imu_angleZ_right',
       'imu_angleX_left', 'imu_angleY_left', 'imu_angleZ_left',
       'imu_angleX_spine', 'imu_angleY_spine', 'imu_angleZ_spine',
       'imu_angularX_left', 'imu_angularY_left', 'imu_angularZ_left',
       'imu_angularX_right', 'imu_angularY_right', 'imu_angularZ_right',
       'imu_angularX_spine', 'imu_angularY_spine', 'imu_angularZ_spine', 'PD'],
      dtype='object')

In [32]:
# Creamos una lista de los valores únicos de fecha como strings
fechas_unicas = data['date_measure'].astype(str).unique()
# 'fechas_unicas' contiene una lista de las fechas únicas encontradas en 'date_measure' como strings
print(fechas_unicas)

['2023-9-29-11-53-21' '2023-6-22-10-28-8' '2023-6-16-14-45-43'
 '2023-9-19-10-12-28' '2023-3-23-13-14-7' '2023-7-6-12-7-50'
 '2023-11-7-15-14-42' '2023-2-23-15-58-22' '2023-9-30-8-42-24'
 '2023-9-30-8-58-32' '2024-1-16-16-1-25' '2023-6-16-9-48-30'
 '2023-6-17-8-26-3' '2023-3-17-11-12-26' '2023-3-18-13-33-51'
 '2023-6-17-8-39-41' '2023-3-17-11-1-26' '2023-6-16-14-38-22'
 '2023-10-5-10-2-44' '2023-3-18-10-50-25' '2023-7-6-9-33-42'
 '2023-9-5-12-31-47' '2024-2-2-9-34-54' '2023-2-23-15-37-18'
 '2023-6-29-10-40-43' '2023-6-22-9-54-43' '2023-9-30-9-41-22'
 '2023-9-30-12-3-43' '2023-3-23-10-28-48' '2024-2-2-11-11-47'
 '2024-1-30-9-6-49' '2024-1-30-10-17-23' '2024-2-15-8-52-21'
 '2024-2-15-10-1-41' '2024-2-15-11-18-44' '2023-6-17-9-42-57'
 '2023-8-31-9-14-29' '2023-2-23-14-11-53' '2024-2-13-15-13-47'
 '2023-9-30-12-28-15' '2023-9-7-12-2-5' '2023-2-21-15-8-52'
 '2023-11-21-9-17-56' '2023-9-22-14-56-19' '2023-4-11-8-49-25'
 '2023-3-30-8-35-11' '2023-11-3-10-52-41' '2023-6-16-11-54-33'
 '2023-9-5

# Feature Engineering Stage 2: Statistical measures of frequency domain

In the stage 2 of the feature engineering, we will apply the fourier transform to the windows and then build a total of 15 simple statistical features —
1. mean
2. standard deviation
3. average absolute deviation
4. minimum value
5. maximum value
6. difference of maximum and minimum values
7. median
8. median absolute deviation
9. interquartile range
10. negative values count
11. positive values count
12. number of values above mean
13. number of peaks
14. skewness
15. kurtosis

Definir la función para calcular las medidas estadísticas de una ventana

In [33]:
def calcular_medidas_estadisticas(columna):
    #print(columna)
    medidas = {
        'mean': np.mean(columna),
        'std': np.std(columna),
        'mean_abs_dev': np.mean(np.abs(columna - np.mean(columna))),
        'min': np.min(columna),
        'max': np.max(columna),
        'range': np.max(columna) - np.min(columna),
        'median': np.median(columna),
        'median_abs_dev': np.median(np.abs(columna - np.median(columna))),
        'interquartile_range': np.percentile(columna, 75) - np.percentile(columna, 25),
        'negative_count': np.sum(columna < 0),
        'positive_count': np.sum(columna > 0),
        'above_mean_count': np.sum(columna > np.mean(columna)),
        'local_maxima_count': len(columna) - np.sum((columna.shift(-1) < columna) & (columna.shift(1) < columna)),
        'skewness': skew(columna),
        'kurtosis': kurtosis(columna)
    }
    #print(medidas)
    return medidas

Crear un nuevo DataFrame para almacenar los datos de las ventanas

In [34]:
columnas_sensores = ['imu_gyroX_right', 'imu_gyroY_right', 'imu_gyroZ_right', 'imu_accX_right', 'imu_accY_right', 'imu_accZ_right', 'imu_gyroX_left', 'imu_gyroY_left', 'imu_gyroZ_left', 'imu_accX_left', 'imu_accY_left', 'imu_accZ_left', 'imu_gyroX_spine', 'imu_gyroY_spine', 'imu_gyroZ_spine', 'imu_accX_spine', 'imu_accY_spine', 'imu_accZ_spine', 'imu_angleX_right', 'imu_angleY_right', 'imu_angleZ_right', 'imu_angleX_left', 'imu_angleY_left', 'imu_angleZ_left', 'imu_angleX_spine', 'imu_angleY_spine', 'imu_angleZ_spine', 'imu_angularX_left', 'imu_angularY_left', 'imu_angularZ_left', 'imu_angularX_right', 'imu_angularY_right', 'imu_angularZ_right', 'imu_angularX_spine', 'imu_angularY_spine', 'imu_angularZ_spine']
medidas_estadisticas = ['mean', 'std', 'mean_abs_dev',
                      'min', 'max', 'range', 'median', 'median_abs_dev',
                      'interquartile_range', 'negative_count', 'positive_count', 'above_mean_count',
                      'local_maxima_count', 'skewness', 'kurtosis']
def generar_nombres_columnas(columnas_sensores, medidas_estadisticas):
    nombres_columnas = ['anon_id', 'date_measure', 'window_number', 'first_timestamp']
    for columna in columnas_sensores:
        for medida in medidas_estadisticas:
            nombre_columna = f"fft_{columna}_{medida}"
            nombres_columnas.append(nombre_columna)
    nombres_columnas.append("PD")
    return nombres_columnas

columnas_df_final=generar_nombres_columnas(columnas_sensores,medidas_estadisticas)
print(columnas_df_final)

['anon_id', 'date_measure', 'window_number', 'first_timestamp', 'fft_imu_gyroX_right_mean', 'fft_imu_gyroX_right_std', 'fft_imu_gyroX_right_mean_abs_dev', 'fft_imu_gyroX_right_min', 'fft_imu_gyroX_right_max', 'fft_imu_gyroX_right_range', 'fft_imu_gyroX_right_median', 'fft_imu_gyroX_right_median_abs_dev', 'fft_imu_gyroX_right_interquartile_range', 'fft_imu_gyroX_right_negative_count', 'fft_imu_gyroX_right_positive_count', 'fft_imu_gyroX_right_above_mean_count', 'fft_imu_gyroX_right_local_maxima_count', 'fft_imu_gyroX_right_skewness', 'fft_imu_gyroX_right_kurtosis', 'fft_imu_gyroY_right_mean', 'fft_imu_gyroY_right_std', 'fft_imu_gyroY_right_mean_abs_dev', 'fft_imu_gyroY_right_min', 'fft_imu_gyroY_right_max', 'fft_imu_gyroY_right_range', 'fft_imu_gyroY_right_median', 'fft_imu_gyroY_right_median_abs_dev', 'fft_imu_gyroY_right_interquartile_range', 'fft_imu_gyroY_right_negative_count', 'fft_imu_gyroY_right_positive_count', 'fft_imu_gyroY_right_above_mean_count', 'fft_imu_gyroY_right_local_m

In [35]:
df_ventanas_fft = pd.DataFrame(columns=columnas_df_final)

Definir los parámetros para las ventanas

In [36]:
frecuencia_muestreo = 50  # Hz
longitud_ventana_s = 1  # segundos
overlap = 0.5  # 50%

Calcular la cantidad de filas por ventana y el paso de tiempo entre ventanas

In [37]:
filas_por_ventana = int(longitud_ventana_s * frecuencia_muestreo)
paso_tiempo = int(filas_por_ventana * (1 - overlap))

In [38]:
# Ignorar todos los warnings
warnings.filterwarnings("ignore")

In [39]:
columnas_df_final_sensores=columnas_df_final
columnas_df_final_sensores.remove('anon_id')
columnas_df_final_sensores.remove('date_measure')
columnas_df_final_sensores.remove('window_number')
columnas_df_final_sensores.remove('first_timestamp')
columnas_df_final_sensores.remove('PD')
print(columnas_df_final_sensores)

['fft_imu_gyroX_right_mean', 'fft_imu_gyroX_right_std', 'fft_imu_gyroX_right_mean_abs_dev', 'fft_imu_gyroX_right_min', 'fft_imu_gyroX_right_max', 'fft_imu_gyroX_right_range', 'fft_imu_gyroX_right_median', 'fft_imu_gyroX_right_median_abs_dev', 'fft_imu_gyroX_right_interquartile_range', 'fft_imu_gyroX_right_negative_count', 'fft_imu_gyroX_right_positive_count', 'fft_imu_gyroX_right_above_mean_count', 'fft_imu_gyroX_right_local_maxima_count', 'fft_imu_gyroX_right_skewness', 'fft_imu_gyroX_right_kurtosis', 'fft_imu_gyroY_right_mean', 'fft_imu_gyroY_right_std', 'fft_imu_gyroY_right_mean_abs_dev', 'fft_imu_gyroY_right_min', 'fft_imu_gyroY_right_max', 'fft_imu_gyroY_right_range', 'fft_imu_gyroY_right_median', 'fft_imu_gyroY_right_median_abs_dev', 'fft_imu_gyroY_right_interquartile_range', 'fft_imu_gyroY_right_negative_count', 'fft_imu_gyroY_right_positive_count', 'fft_imu_gyroY_right_above_mean_count', 'fft_imu_gyroY_right_local_maxima_count', 'fft_imu_gyroY_right_skewness', 'fft_imu_gyroY_ri

In [40]:
# Iterar sobre los datos originales en el DataFrame
window_number = 1
for i in range(0, len(data) - filas_por_ventana + 1, paso_tiempo):
    # Seleccionar la primera fila de cada ventana
    primera_fila_ventana = data.iloc[i]

    # Seleccionar los valores relevantes de la fila
    anon_id = primera_fila_ventana['anon_id']
    date_measure = primera_fila_ventana['date_measure']
    PD = primera_fila_ventana['PD']
    first_timestamp = primera_fila_ventana['time_stamp']

    # Seleccionar la ventana incluyendo tanto los valores de los sensores como los metadatos
    ventana = data.iloc[i:i + filas_por_ventana]

    # Verificar si los valores en la primera y última fila de la ventana son iguales
    while not ventana.iloc[0][['anon_id', 'date_measure', 'PD']].equals(
            ventana.iloc[-1][['anon_id', 'date_measure', 'PD']]):
        ventana = ventana.iloc[:-1]  # Reducir la longitud de la ventana eliminando la última fila

    # Eliminar los valores de 'anon_id', 'date_measure' y 'PD' de la ventana antes de calcular las medidas estadísticas
    ventana_valores_sensor = ventana.drop(['anon_id', 'date_measure', 'time_stamp', 'PD'], axis=1)
    ventana_fft = np.fft.fft(ventana_valores_sensor)
    ventana_fft_abs = np.abs(ventana_fft)  # Aplicar valor absoluto a la transformada de Fourier
    ventana_fft_df = pd.DataFrame(ventana_fft_abs, columns=ventana_valores_sensor.columns)


# Calcular las medidas estadísticas de la ventana para cada columna
    datos_ventana = {
        'anon_id': anon_id,
        'date_measure': date_measure,
        'window_number': window_number,
        'first_timestamp': first_timestamp,
        'PD': PD
    }
    for sensor_col in ventana_valores_sensor.columns:
        medidas = calcular_medidas_estadisticas(ventana_fft_df[sensor_col])
        for medida, medida_valor in medidas.items():
            nombre_columna = f"fft_{sensor_col}_{medida}"
            datos_ventana[nombre_columna] = medida_valor

    # Añadir los datos de la ventana al DataFrame
    df_ventanas_fft = pd.concat([df_ventanas_fft, pd.DataFrame(datos_ventana, index=[0])], ignore_index=True)

    window_number += 1


In [41]:
# Restaurar el comportamiento predeterminado de los warnings
warnings.filterwarnings("default")

In [42]:
# Mostrar las primeras filas del DataFrame de ventanas
print(df_ventanas_fft.head())

  anon_id        date_measure window_number  first_timestamp  \
0       9  2023-9-29-11-53-21             1              0.0   
1       9  2023-9-29-11-53-21             2            532.0   
2       9  2023-9-29-11-53-21             3           1029.0   
3       9  2023-9-29-11-53-21             4           1587.0   
4       9  2023-9-29-11-53-21             5           2121.0   

   fft_imu_gyroX_right_mean  fft_imu_gyroX_right_std  \
0               4492.082286              2398.058032   
1               4264.240537              2839.961398   
2               4044.778170              3058.455093   
3               3271.885272              2020.474359   
4               4497.486671              3248.411077   

   fft_imu_gyroX_right_mean_abs_dev  fft_imu_gyroX_right_min  \
0                       1853.206304               351.967162   
1                       2047.382986                16.400323   
2                       2368.328112                16.400323   
3                     

In [43]:
df_ventanas_fft.to_pickle(ROOT_DIR / 'data' / 'processed'/'anon_imu_data_fft_features.pkl')
df_ventanas_fft.to_csv(ROOT_DIR / 'data' / 'processed'/'anon_imu_data_fft_features.csv')

In [44]:
#df_ventanas: pd.DataFrame = pd.read_pickle(ROOT_DIR / 'data' / 'processed' / 'anon_imu_data_features.csv')